In [ ]:
from sr.volvox import analyze, load_data
from sr.volvox.io import get_xyz_data
from sr.volvox.process import get_orientation_vec
from pathlib import Path
import json
import pandas as pd

import matplotlib.pylab as plt
from sr.plot_util import *
import numpy as np

In [ ]:
results = pd.read_hdf('results2p.h5')
results["R"] = 8.0


results['phase_lag_r'] = results.phase_lack
# m = results.run_type=='final_run_wave_physical2'
sel = results.query('(alpha != 0 or alpha != 180) and (run_type=="final_run_wave_physical2" or run_type=="final_run_wave_physical") ')

results.loc[sel.index, 'phase_lag_r'] = sel.k_theta/np.tan(sel.alpha.astype('float')/180*np.pi)*np.pi/4

#Add a minus sign, to have simplectic waves positive and anti negative
results['phase_lag_r'] = (-results.phase_lag_r + np.pi) % (2 * np.pi) - np.pi

In [ ]:

from scipy.optimize import curve_fit

def f(t, alpha, omega_n, kappa):
    A = np.sin(alpha)
    kappa = np.abs(kappa)
    return (np.cos(omega_n*t)*A**2 + (1-A**2))*np.exp(-kappa*t)

def get_tangent_correlation(df, smooth=None):
    if not smooth:
        smooth = int(df.tau/2.5)
    data = load_data(Path(df.name))
    T = data[['x', 'y', 'z']].rolling(smooth).mean().diff()
    T = T.div(np.sqrt((T*T).sum(axis=1)),axis=0)
    q = {}
    for i in range(int(T.last_valid_index()/df.tau)):
        ts = i*df.tau
        q[ts] = ((T*T.shift(int(ts/2.5))).dropna()).sum(axis=1).mean()
    return pd.Series(q).dropna()

def get_angular_correlation(df):
    data = load_data(Path(df.name))
    T = data[['nx', 'ny', 'nz']]
    T = T.div(np.sqrt((T*T).sum(axis=1)),axis=0)
    q = {}
    for i in range(int(T.last_valid_index()/df.tau)):
        ts = i*df.tau
        q[ts] = ((T*T.shift(int(ts/2.5))).dropna()).sum(axis=1).mean()
    return pd.Series(q)

def fit_angular_correlation(q, fix_kappa=None):
    F = np.fft.rfftfreq(len(q))
    w = F[np.abs(np.fft.rfft(q - q.mean())).argmax()]*2*np.pi
    alpha, omega_n = 0.0, w
    #q = q.loc[:q.last_valid_index()/1.5]
    
    kappa = fix_kappa if fix_kappa else 0.0
    err_exp = 0.5
    try:
        (alpha, omega_n), pcov = curve_fit(lambda t, a, o: f(t, a, o, kappa),
                                       q.index,q.values, sigma=(1+q.index)**err_exp,
                                           p0=[alpha, omega_n], maxfev=int(1e4))
    except RuntimeError:
        raise
    if fix_kappa is None:
        (alpha, omega_n, kappa), pcov = curve_fit(f, q.index,q.values,
                                                 p0=[alpha, omega_n, 0.0],  
                                                  sigma=(1+q.index)**err_exp,
                                                  maxfev=int(1e4))

    #,method='trf',                      #bounds=[(0,0,-np.inf),(np.pi, np.inf,np.inf)], 

    alpha = np.abs((alpha + np.pi) % (2*np.pi) - np.pi)
    kappa = np.abs(kappa)
    return alpha, np.abs(omega_n), kappa, pcov


def compare_fit(df):
    
    q = correlation_fits.get(df.name)
    f_alpha, f_omega, f_kappa, pcov = fit_angular_correlation(q)
    l, = plt.plot(q.index, f(q.index, f_alpha, f_omega, f_kappa), '--', 
             label="normal vec.")
    plt.scatter(q.index,q.values,s=2,c=l.get_color())
    print(q.index[-1])
    l, = plt.plot(q.index, f(q.index, df.c_S_alpha, df.c_S_omega, df.c_S_kappa), '--',
             label=r"$0.75 T_s$ normal vec.") 
    
    q = correlation_fits_T.get(df.name)
    f_alpha, f_omega, f_kappa, pcov = fit_angular_correlation(q)
    plt.title(f"$\chi={df.phase_lag_r/np.pi*180:.0f}^\circ$ $k_\\varphi={df.k_theta}$ $\\theta_r={df.thetaZ/np.pi*180:.0f}^\circ$")
    
    l, = plt.plot(q.index, f(q.index, f_alpha, f_omega, f_kappa), '--',
             label="trangent traj.")              
    plt.scatter(q.index,q.values,s=2,c=l.get_color())
    
    
    
    #plt.ylim(-1.2,1.2)
    plt.legend(loc=3, ncol=2,)
    plt.xlabel('$\\tau / \\tau_b$')
    plt.ylabel('auto correlation')
    
eta = 15
R = 8  
d_r = 1/(8*np.pi*eta*R**3)

In [ ]:
assert False # Results saved in results
#for name, df in results.query('run_type=="final_long_smooth"').iterrows():
correlation_fits_T = {}
results['c_t_alpha'] = None
results['c_t_omega'] = None
results['c_t_kappa'] = None
results['t_pcov'] = None

results['c_t_S_alpha'] = None
results['c_t_S_omega'] = None
results['c_t_S_kappa'] = None
results['t_S_pcov'] = None

correlation_fits = {}
results['c_alpha']  = None
results['c_omega']  = None
results['c_kappa']  = None
results['pcov']  = None

results['c_S_alpha']  = None
results['c_S_omega']  = None
results['c_S_kappa']  = None
results['S_pcov']  = None


for name, df in results.query('run_type=="final_long_smooth" and num_cil_eq==7').iterrows():
    q = get_tangent_correlation(df)
    q.index = q.index /df.tau
    a = fit_angular_correlation(q)
    results.loc[name, ['c_t_alpha', 'c_t_omega', 'c_t_kappa', 't_pcov']] = a
    
    q2 = q.loc[:q.last_valid_index()/1.5]
    a = fit_angular_correlation(q2)
    results.loc[name, ['c_t_S_alpha', 'c_t_S_omega', 'c_t_S_kappa', 't_S_pcov']] = a
    
    correlation_fits_T[name] = q

    
    q = get_angular_correlation(df)
    q.index = q.index /df.tau
    a = fit_angular_correlation(q)
    results.loc[name, ['c_alpha', 'c_omega', 'c_kappa', 'pcov']] = a
    
    q2 = q.loc[:q.last_valid_index()/1.5]
    a = fit_angular_correlation(q2)
    results.loc[name, ['c_S_alpha', 'c_S_omega', 'c_S_kappa', 'S_pcov']] = a
    
    correlation_fits[name] = q
    
import pickle
with open('correlation_fits.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(correlation_fits, f, pickle.HIGHEST_PROTOCOL)
    
with open('correlation_fits_T.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(correlation_fits_T, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
results['c_t_alpha_err'] = None
results['c_t_omega_err'] = None
results['c_t_kappa_err'] = None

results['c_alpha_err'] = None
results['c_omega_err'] = None
results['c_kappa_err'] = None


for name, df in results.query('run_type=="final_long_smooth"').iterrows():
    if results.loc[name].pcov is not None:
        perr = np.sqrt(np.diag(results.loc[name].pcov))
        results.loc[name, ['c_alpha_err', 'c_omega_err', 'c_kappa_err']] = perr    

    if results.loc[name].t_pcov is not None:
        perr = np.sqrt(np.diag(results.loc[name].t_pcov))
        results.loc[name, ['c_t_alpha_err', 'c_t_omega_err', 'c_t_kappa_err']] = perr

In [ ]:
import pickle
with open('correlation_fits.pickle', 'rb') as filep:
    correlation_fits = pickle.load(filep)
    
with open('correlation_fits_T.pickle', 'rb') as filep:
    correlation_fits_T = pickle.load(filep)


## Correlation Parameter (Fig. 6 Fig. 11)

In [ ]:
fs = (REVT_FULL/2, 2.1)

In [ ]:
fig = plt.figure(figsize=fs)
ax = plt.subplot()
sel = results.query('run_type=="final_long_smooth" and num_cil_eq==7 and thetaZ==0.0').sort_values('k_theta')
for k_theta, df in sel.groupby('k_theta'):
    df = df.sort_values('phase_lag_r')
    err = np.abs(df.c_kappa - df.c_S_kappa)/d_r/df.tau/2 
    df = df[err < 0.2]
    p = plt.errorbar(df.phase_lag_r, np.abs(df.c_kappa)/d_r/df.tau/2, df.c_kappa_err/d_r/df.tau/2, 
                 marker='o',
                 label=f"$k_\\varphi = {k_theta}$")
    #plt.errorbar(df.phase_lag_r, np.abs(df.c_S_kappa)/d_r/df.tau/2, df.c_kappa_err/d_r/df.tau/2, 
    #             marker='o', color = p[0].get_color(), ls='--',
    #             label=None)

plt.legend(ncol=2)
plt.ylabel(r'$\kappa \tau_{rot}$')
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)
ax.set_xlabel(r"phase lag $\chi$")

#plt.colorbar()   
plt.savefig('figs/c_kappa_k.pdf')

In [ ]:
fig = plt.figure(figsize=fs)
ax = plt.subplot()
sel = results.query('run_type=="final_long_smooth" and num_cil_eq==7 and thetaZ==0.0').sort_values('k_theta')
for k_theta, df in sel.groupby('k_theta'):
    df = df[~(df.c_omega_err >  10/180*np.pi)]
    #df = df[~(df.c_t_omega_err > 0.1)]
    err = np.abs(df.c_omega - df.c_S_omega)
    df = df[err < 10/180*np.pi]
    
    df = df.sort_values('phase_lag_r')
    #plt.plot(df.phase_lag_r, np.abs(df.c_kappa)/d_r/df.tau, 'o-',label=f"$k_\\theta = {k_theta}$")
    p = plt.errorbar(df.phase_lag_r, df.c_omega, df.c_omega_err, marker='o',
                 label=f"$k_\\varphi = {k_theta}$")
    
    #plt.errorbar(df.phase_lag_r, np.abs(df.c_t_omega), df.c_t_omega_err, 
    #             marker='o', color = p[0].get_color(), ls='--',
    #             label=None)
#plt.title("n==7, smooth")
#plt.legend(ncol=2)
plt.ylabel(r'$\Omega_c \tau_b$')
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)
set_angles_y(ax, angles=np.linspace(-0, 25,5), ax_in_rad=True)

ax.set_xlabel(r"phase lag $\chi$")

#plt.colorbar()   
plt.savefig('figs/c_omega_k.pdf')

In [ ]:
err

In [ ]:
fig = plt.figure(figsize=fs)
ax = plt.subplot()
sel = results.query('run_type=="final_long_smooth" and num_cil_eq==7 and thetaZ==0.0').sort_values('k_theta')
mask = sel.c_alpha >np.pi/2
sel.loc[mask,'c_alpha'] = (sel.loc[mask,'c_alpha'] - np.pi)*-1
mask = sel.c_S_alpha >np.pi/2
sel.loc[mask,'c_S_alpha'] = (sel.loc[mask,'c_S_alpha'] - np.pi)*-1
for k_theta, df in sel.groupby('k_theta'):
    df = df[~(df.c_alpha_err > 25/180*np.pi)]
    err = np.abs(df.c_alpha - df.c_S_alpha)
    df = df[err <25/180*np.pi]
    df = df.sort_values('phase_lag_r')
    #plt.plot(df.phase_lag_r, np.abs(df.c_kappa)/d_r/df.tau, 'o-',label=f"$k_\\theta = {k_theta}$")
    plt.errorbar(df.phase_lag_r, df.c_alpha, df.c_alpha_err, marker='o',
                 label=f"$k_\\varphi = {k_theta}$")
#plt.title("n==7, smooth")
#plt.legend(ncol=2)
plt.ylabel(r'$\alpha $')
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)
set_angles_y(ax, angles=np.linspace(0, 90,5), ax_in_rad=True)

ax.set_xlabel(r"phase lag $\chi$")

#plt.colorbar()   
plt.savefig('figs/c_alpha_k.pdf')

In [ ]:
fig = plt.figure(figsize=fs)
ax = plt.subplot()
sel = results.query('run_type=="final_long_smooth" and num_cil_eq==7 and k_theta==0.0').sort_values('k_theta')
sel['thetaZb'] = round(sel.thetaZ.astype(float)/np.pi*180.)
for thetaZ, df in sel.groupby('thetaZb'):
    err = np.abs(df.c_kappa - df.c_S_kappa)/d_r/df.tau/2 
    df = df[err < 0.2]
    df = df.sort_values('phase_lag_r')
    plt.errorbar(df.phase_lag_r, np.abs(df.c_kappa)/d_r/df.tau/2, df.c_kappa_err/d_r/df.tau/2, 
                 marker='o',
                label=f"$\\theta_r = {thetaZ}^\\circ$")

#plt.title("n==7, smooth")
plt.legend(ncol=2, labelspacing=0.2, fontsize=7)
plt.ylabel(r'$\kappa \tau_{rot}$')

set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)
plt.ylim(0,0.6)
ax.set_xlabel(r"phase lag $\chi$")
#plt.xlim(-1.3*np.pi,np.pi)
plt.savefig('figs/c_kappa_thetaR.pdf')

In [ ]:
fig = plt.figure(figsize=fs)
ax = plt.subplot()
sel = results.query(
    'run_type=="final_long_smooth" and num_cil_eq==7 and k_theta==0.0').sort_values('k_theta')
sel['thetaZb'] = round(sel.thetaZ.astype(float)/np.pi*180.)
for thetaZ, df in sel.groupby('thetaZb'):
    df = df[~(df.c_omega_err > 5/180*np.pi)]
    err = np.abs(df.c_omega - df.c_S_omega)
    df = df[err < 5/180*np.pi]
    df = df.sort_values('phase_lag_r')
    plt.errorbar(df.phase_lag_r, df.c_omega, df.c_omega_err,
                 marker="o",
                label=f"$\\theta_r = {thetaZ}^\\circ$")
#plt.title("n==7, smooth")
#plt.legend(ncol=2, labelspacing=0.2)
plt.ylabel(r'$\Omega_c \tau$')
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)
set_angles_y(ax, angles=np.linspace(-1, 2,5), ax_in_rad=True)

ax.set_xlabel(r"phase lag $\chi$")
plt.savefig('figs/c_omega_thetaR.pdf')

In [ ]:
fig = plt.figure(figsize=fs)
ax = plt.subplot()
sel = results.query(
    'run_type=="final_long_smooth" and num_cil_eq==7 and k_theta==0.0').sort_values('k_theta')
sel['thetaZb'] = round(sel.thetaZ.astype(float)/np.pi*180.)
mask = sel.c_alpha >np.pi/2
sel.loc[mask,'c_alpha'] = (sel.loc[mask,'c_alpha'] - np.pi)*-1
mask = sel.c_S_alpha >np.pi/2
sel.loc[mask,'c_S_alpha'] = (sel.loc[mask,'c_S_alpha'] - np.pi)*-1
for thetaZ, df in sel.groupby('thetaZb'):
    df = df[~(df.c_alpha_err > 25/180*np.pi)]
    err = np.abs(df.c_alpha - df.c_S_alpha)
    df = df[err <25/180*np.pi]
    df = df.sort_values('phase_lag_r')
    plt.errorbar(df.phase_lag_r, df.c_alpha, df.c_alpha_err,
                 marker='o',
                 label=f"$\\theta_r = {thetaZ}^\\circ$")
    
#plt.title("n==7, smooth")
#plt.legend(ncol=3, labelspacing=0.2)
plt.ylabel(r'$\alpha$')
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)
set_angles_y(ax, angles=np.linspace(-0, 45,5), ax_in_rad=True)

ax.set_xlabel(r"phase lag $\chi$")
plt.savefig('figs/c_alpha_thetaR.pdf')

## Appendix Figures

In [ ]:
sel = results.query('run_type=="final_long_smooth" and num_cil_eq==7 and thetaZ==0.0').sort_values('k_theta')


In [ ]:
compare_fit(sel.iloc[0])

In [ ]:
compare_fit(sel.iloc[6])

## Figure 8

In [ ]:
fig = plt.figure(figsize=(REVT_HALF, 2.4))
ax = plt.subplot()
sel = results.query('run_type=="final_long_smooth" and num_cil_eq==7 and thetaZ==0.0').sort_values('k_theta')
for k_theta, df in sel.groupby('k_theta'):
    df = df[~(df.c_omega_err >  10/180*np.pi)]
    #df = df[~(df.c_t_omega_err > 0.1)]
    err = np.abs(df.c_omega - df.c_S_omega)
    df = df[err < 10/180*np.pi]
    
    df = df.sort_values('phase_lag_r')
    #plt.plot(df.phase_lag_r, np.abs(df.c_kappa)/d_r/df.tau, 'o-',label=f"$k_\\theta = {k_theta}$")
    plt.plot(df.phase_lag_r, df.c_omega/np.abs(df.Omega_n_mean*df.tau),"o-",
                 label=f"$k_\\varphi = {k_theta}$")
    
    #    p = plt.errorbar(df.phase_lag_r, df.c_omega, df.c_omega_err, marker='o',
    
    #plt.errorbar(df.phase_lag_r, np.abs(df.c_t_omega), df.c_t_omega_err, 
    #             marker='o', color = p[0].get_color(), ls='--',
    #             label=None)
#plt.title("n==7, smooth")
#plt.legend(ncol=2)
plt.ylabel(r'$\Omega_c / |\Omega_n|$')
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)

ax.set_xlabel(r"phase lag $\chi$")

#plt.colorbar()   
plt.savefig('figs/c_omega_vs_omega_n.pdf')

## Helix Radius Figures

In [ ]:
results['helix_R'] = None
results['helix_pitch'] = None

results['helix_R_t'] = None
results['helix_pitch_t'] = None


for name, df in results.query('run_type=="final_long_smooth" and num_cil_eq==7').iterrows():
    v = df.v_normal_m*df.tau/df.R
    omega = df.c_t_omega
    alpha = df.c_t_alpha
    if alpha >np.pi/2:
        alpha = (alpha - np.pi)*-1
    helix_pitch_t = v*np.cos(alpha)/omega
    helix_R_t = v*np.sin(alpha)/omega/2/np.pi
    results.loc[name, ['helix_R_t', 'helix_pitch_t']] = [helix_R_t, helix_pitch_t]
    
    omega = df.c_omega
    alpha = df.c_alpha
    if alpha >np.pi/2:
        alpha = (alpha - np.pi)*-1
    helix_pitch = v*np.cos(alpha)/omega
    helix_R = v*np.sin(alpha)/omega/2/np.pi
    results.loc[name, ['helix_R', 'helix_pitch']] = [helix_R, helix_pitch]
    
    


In [ ]:
fig = plt.figure(figsize=fs)
ax = plt.subplot(111)
color_cycle = ax._get_lines.prop_cycler
sel = results.query('run_type=="final_long_smooth" and num_cil_eq==7 and k_theta==0.0').sort_values('thetaZ')
sel['thetaZb'] = round(sel.thetaZ.astype(float)/np.pi*180.)

for thetaZ, df in sel.groupby('thetaZb'):
    df = df[~(df.c_omega_err > 5/180*np.pi)]
    err = np.abs(df.c_omega - df.c_S_omega)
    df = df[err < 5/180*np.pi]
    
    df = df[~(df.c_alpha_err > 25/180*np.pi)]
    err = np.abs(df.c_alpha - df.c_S_alpha)
    df = df[err <25/180*np.pi]
    
    df = df.sort_values('phase_lag_r')
    if(len(df)) < 3: 
        next(color_cycle)
        continue
    l, = ax.plot(df.phase_lag_r, df.helix_R, 'o-',
                 label=f"$k_\\varphi = {k_theta}$")
    ax.plot(df.phase_lag_r, df.helix_R_t, 'o--', c = l.get_color(),
            )

#plt.title("n==7, smooth")
#plt.legend(ncol=2)
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)

ax.set_xlabel(r"phase lag $\chi$")

ax.set_ylabel('Helix Radius $R_h / R$')
#plt.legend(loc=(0.55,0.55))

#plt.tight_layout()
#plt.colorbar()   
plt.savefig('figs/helix_radius_thetaR.pdf')

In [ ]:
fig = plt.figure(figsize=fs)
ax = plt.subplot(111)

sel = results.query('run_type=="final_long_smooth" and num_cil_eq==7 and thetaZ==0.0').sort_values('k_theta')

for k_theta, df in sel.groupby('k_theta'):
    df = df[~(df.c_omega_err > 5/180*np.pi)]
    err = np.abs(df.c_omega - df.c_S_omega)
    df = df[err < 5/180*np.pi]
    
    df = df[~(df.c_alpha_err > 25/180*np.pi)]
    err = np.abs(df.c_alpha - df.c_S_alpha)
    df = df[err <25/180*np.pi]
    if(len(df)) < 3: 
        next(color_cycle)
        continue
    
    df = df.sort_values('phase_lag_r')
    l, = ax.plot(df.phase_lag_r, df.helix_R, 'o-',
                 label=f"$k_\\varphi = {k_theta}$")

#plt.title("n==7, smooth")
#plt.legend(ncol=2)
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)
ax.set_yticks([0,5,10])
ax.set_xlabel(r"phase lag $\chi$")

ax.set_ylabel('Helix Radius $R_h / R$')
#plt.legend(loc=(0.55,0.55))

#plt.tight_layout()
#plt.colorbar()   
plt.savefig('figs/helix_radius_k.pdf')